In [1]:
# -*- coding: utf-8 -*-
import dataiku
import pandas as pd, numpy as np
from dataiku import pandasutils as pdu

# Read recipe inputs
BY_ACCOUNT = dataiku.Dataset("BY_ACCOUNT")
BY_ACCOUNT_df = BY_ACCOUNT.get_dataframe()

df = BY_ACCOUNT_df

In [5]:
customers = df.CUSTOMER.unique()
len(customers)

657099

In [0]:
import string
import nltk
from nltk.tag import pos_tag

df_dict = pd.DataFrame(nltk.corpus.words.words(), columns=['word'])
df_dict['word'] = df_dict['word'].str.upper()

idx = 0

for c in customers:
    
    c_str = c.translate(str.maketrans('', '', string.punctuation))
    f = c_str.split()
    
    for w in f:
        is_in_dict = len(df_dict[df_dict['word']==w])>0
        print(w, is_in_dict)
    
    print()
    
    if idx>10:
        break;

BERGE False
TRANSPORT True

COUNTY True
OF True
MOBILE True

IEC False
INC False

BUYBACKBOY False
LLC False

PJF False
ENTERPRISES False
INC False

G True
A True
ADVERTISING True

BAUMGARDNER False
MOTORS False
INC False

MULTITECH False
SEALERS False
INC False

SINAK False
PLUMBING True
CO False
INC False

TRICOUNTY False
ENGINEERING True
INC False

THE True
ELLIOTT True
COMPANY True

SERIOUS True
GRIP True
AND True
ELECTRIC True

LT False
FOOD True
DISTRIBUTOR True
INC False

WALL True
FX False
LTD False

PERFORMANCE True
TESTING True

HERITAGE True
HAVEN True
INC False

INSANELY True
FAST True
HOME True
SOLUTIONS False
INC False

S True
K True
SANITATION True
INC False

TK False
VISION True
HOLDINGS False
INCORPORATED True

PAYNE False
R True
K True
REFRIGERATION True
ELECTRICAL True
INC False

MICRO True
INFORMATICA False
LLC False

MARATHON True
FLORIST True
INC False

SPINNING True
CRANE True
WORKS True
LLC False

FALLS False
METAL True
FABRICATORS False
INDUSTRIAL True
SERVICES

MIRROR True
DESIGNS False
INC False

NEWWAY False
CONCRETE True
FORMING True
LTD False

METRO False
MANAGEMENT True
LLC False

GIBELLINO False
CONSTRUCTION True
CO False
INC False

SEQUOIA True
CONSULTANTS False
INC False

DR False
GREEN True
THUMB True
LANDSCAPING False

DD False
ELECTRIC True

METAL True
FAB False
ENGINEERING True
INC False

TRI True
PROPERTIES False
LLC False

BOXX False
MOVING True
STORAGE True
LLC False

ETC False
ECO False
TREE True
CARE True
INC False

SKY True
HELICOPTERS False
INC False

SAGOLA False
CONTRACTING False
INC False

KEMCO False
FACILITIES False
SERVICES False
LLC False

LAMS False
CONSTRUCTION True

MYERS False
WAYNE True
HARDWOOD True
FLOORS False

WELCH False
FUNERAL True
HOME True

KALAMAZOO False
VALLEY True
PLANT True
GROWERS False
COOPERATIVE False
INC False

EASTERN True
ARIZONA True
SURFACE True
UNDERGROUND True
INC False

CLARKE False
INCORPORATE True
LLC False

49 False
NORTH True
MECHANICAL True
LTD False

W True
R True
PROPERTIES False

OF True
DEER True
TRAIL True

TOWN True
OF True
DEERTRAIL False

ASPHALT True
EQUIPMENT True
SERVICE True
CO False

NAPOLI False
MARBLE True
AND True
GRANITE True
CORP True

ST True
CLAIRE True
MEDICAL True
CENTER True
INC False

INELLO False
ELECTRIC True
LLC False

CONDUCTOR True
POWER True
LLC False

ELITE True
AUTO True
SERVICE True

DSI False
TECHNOLOGIES False
LLC False

WILLS False
TAXI True
SERVICE True

OVERHEAD True
TECHNOLOGY True
INC False

DEL True
FIACCOS False
SEWERS False
INC False

THE True
DAMON True
CO False

THE True
DAMON True
COMPANY True

FIBER True
1 False
COMMUNICATIONS False
INC False

SAFETY True
CARE True
TRANSPORTATION True
INC False

US True
PLY True
INC False

KILLAWOG False
FIRE True
DISTRICT True

1720757 False
ALBERTA True
LTD False

JACKS False
AC False
REFRIGERATION True
LLC False

BRYANTS False
HEATING True
COOLING True
INC False

THE True
LAW True
OFFICE True
OF True
EBONY True
W True
RODGERS False
LLC False

STARMOUNT False
CLEANERS False
INC Fals

SERVICE True
INC False

RAPHAEL True
SAINT True
ROOFING True
INC False

VILLAGE True
OF True
HARRISBURG False

ST True
PETERS False
HOSPITAL True

ACE True
PAINTING True
SERVICE True
INC False

COPICOM False
INC False

WOOSTER False
PLUMBING True
HEATING True
CO False

AIRE True
TECH True
HEATING True
AIR True
CONDITIONING False

RUMBO True
PET True
SUPPLIES False

SPRINGS False
TREE True
LANDSCAPING False

SPRINGS False
TREE True
LANDSCAPE True
SERVICES False

3 False
P True
L True
WORLDWIDE False
INC False

Z True
B True
WISCO False
INC False

HOUSE True
OF True
AUTOMATION False
INC False

DEMBY False
SON True
FUNERAL True
HOME True
INC False

WAYNE True
ELECTRIC True
MOTOR True
PUMP True
INC False

J True
P True
AIR True
CONDITIONING False
INC False

BOINC False
LLC False

BEYOND True
CONTRACTING False
LLC False

DANCE True
MAKERS False

ARNOLD True
AIR True
INC False

PACIFIC True
SALES False
SERVICE True
INC False

D True
H True
QUALITY True
CABINETS False

FLEETWASH False
INC Fal

EASTERN True
SHORE True
INC False

TOUNGATE False
WORTH True
HYDROCHEM False
OF True
CORPUS True
CHRISTI False
INC False

GUILLTECH False
LECTRIQUE False
INC False

STUART True
ADAMS False
CORPORATION True

622 False
FX False
SERVICES False
INC False

WOODWARD True
CHILDRENS False
CENTER True
INC False

PILOT True
AIR True
LLC False

ACCURATE True
CLINICAL True
RESEARCH True

KINGS False
KID False
TRUCKING True
LLC False

JEFFREY True
B True
LAMURA False
DDS False
PLLC False

ZIZZO False
RACING True

90915208 False
QUBEC False
INC False

CROSS True
COUNTRY True
ROOFING True
INC False

OLIVER True
SEPTIC True
SERVICE True

DIAMOND True
CONCRETE True
WNY False

COMTEC False
SYSTEMS False
INC False

ZEROREZ False
BOISE False
LLC False

CMART False
SOULTIONS False
LLC False

MOLOKAI False
GENERAL True
HOSPITAL True

LJ False
OMNICO False
AGV False
LLC False

G True
S True
CLEANING True
SERVICES False
INC False

BRUSH True
IN True
A True
RUSH True
PAINTING True
LLC False

V True
F True
R Tr

CONSULTING True
SERVICES False
INC False

RIDGEWAY True
ELECTRIC True
INC False

JALILIE False
ELECTRIC True
INC False

ELDRIDGE False
AIR True
CONDITIONING False
HEATING True
COMPANY True

SCI False
ENGINEERING True
INC False

GPJ False
INVESTMENTS False
LLC False

ARANSAS False
MARINE True
WAYS True
INC False

SCHRAGGER False
SCHRAGGER False
AND True
LAVINE False
PC False

NORTHERN True
TECHNOLOGIES False
LLC False

AIR True
ESSCENTIALS False
INC False

D True
AND True
D True
HELPING True
HANDS False

CE True
COMMUNICATIONS False
INC False

GODDESSWAYINNER False
GODDESS True
LLC False

ALLSTEELHOMECRAFT False
INC False

JOHN True
FERGUSON True
PLUMBING True
HEATING True

ALPINE True
CABINET True
CO False
INC False

ALPINE True
CABINET True
COMPANY True
INC False

EXPRESS True
ENERGY True
SERVICES False
OPERATING True
LP False

CLOSETS False
MORE True
INC False

YOUNGS False
COMMUNICATIONS False
LLC False

JMC False
STONE True
CORP True

J True
M True
C True
STONE True
CORP True

LIFT

SOLAR True
POWER True
LLC False

C True
M True
CONTRACTORS False
INC False

CARTER True
HILL True
CONSTRUCTION True

TRIMAN False
INDUSTRIES False
INC False

MST False
CONCRETE True
PRODUCTS False
INC False

TINDALL False
HAMMOCK True
IRRIGATIO False

DIXIE True
MEMBRANE True
ROOFING True
INC False

AHC False
CONTRACTING False
INC False

WORLDSTAFF False
USA False
CORP True

COLUMBIA True
MALL True
PARTNERS False
LLC False

5GC False
PROPERTIES False
LLC False

KEY True
CARE True
TRANSPORTATION True
LLC False

VALLEY True
ELEVATOR True
INC False

IT True
WORKS True
MILLWORK True
AND True
CABINETRY True
LLC False

911 False
DRY True
OUT True
INC False

EVERYTHINGS False
EVENTFUL True
LLC False

UNION True
PRESBYTERIAN True
SEMINARY True

DREAMERS False
EVENT True
RENTALS False
LLC False

SAMS False
XPRESS False
CAR True
WASH True
LLC False

VICTORY True
DESIGN True
INC False

LANDA False
PROPERTY True
MGMT False

N True
H True
C True
HEALTHCARE False
FARRAGUT False

NHC False
HEALTHCARE

FLORIDA True
DECORATING False
CENTERS False
INC False

OCELLI True
PEST True
SERVICES False
LLC False

PATTI False
WHITE True
REMEDIATION True
SERVICES False
LLC False

A1 False
SCALES True
INC False

MICHAEL True
RAIMO False

HOUSE True
OF True
FLOWERS False
CORP True

CONLEY False
CASTING True
SUPPLY True
CORP True
61 False

SOLID True
GROUND True
PROPERTY True
GROUP True
LLC False

HENDERSON False
TRANSFER True
CO False
INC False

AMY True
FLOWERS False
TEAM True

CLUB True
AUTONEIGE False
RIVE True
SUD True
INC False

MACON True
BEER True
COMPANY True
LLC False

SCHAEFFERS False
TOM True
CAMPING False
AND True
TRAVEL True
CENTER True
INC False

HISPEED False
LOGISTICS True
LLC False

ABSOLUTE True
SOFTWARE False
CORPORATION True

ARSHDEEP False
INC False

SPLASH True
POOL True
SPA True
BUILDERS False
LLC False

DEBRA False
M True
CILIBERTO False
PC False

ARROW True
CHILD True
AND True
FAMILY True
MINISTRIES False

ELITE True
CELL True
REPAIR True

MR True
BIG True
SHOT True
CREDIT

FURNITURE True
CONNECTION True
INC False

JR False
ADELMAN False
CORP True

VIENNA True
TOWNSHIP True
TRUSTEES False

FAST True
GLASS True
SERVICE True
LLC False

POWER True
PEST True
CORP True

POWER True
PEST True
ELIMINATION True
INC False

SALINE True
LANDSCAPE True
CONTRACTORS False

BILLY True
GLENN True
MARSHBURN False

PYRON False
FIRE True
PROTECTION True
INC False

MAJOR True
THEORY True

JML False
LAW True
A True
PROFESSIONAL True
LAW True
CORPORATION True

PARSONAGE True
COTTAGE True

BACH True
SJ False
LTD False

GOOTEE False
CONSTRUCTION True
INC False

GILLESPIE False
ENTERPRISES False
INC False

GILLESPIE False
ENTERPRISES False

MARTINS False
EXCAVATING False
LLC False

PACIFIC True
KITCHEN True
EQUIPMENT True
SERVICE True
INC False

PRECEDENCE True
HOME True
HEALTHCARE False
LLC False

NETTIQUETTE False
VIRTUAL True
LLC False

GARY True
D True
MARTIN True

SUPERSCAPES False
INC False

KINZIE False
CAPITAL True
PARTNERS False
LLC False

PRESCOTT False
BROTHERS False
IN

INC False

BEE True
HAULING False
SERVICES False
INC False

ABLE True
MAINTENANCE True
INC False

JENNIFER True
RENEE False
BROWN True

NORTH True
EAST True
SCHOOL True
DISTRICT True

VITALITY True
HEALTH True
SAFETY True
SERVICES False
LLC False

ALL True
OUT True
MOVING True
STORAGE True
LLC False

BLUE True
MOUNTAIN True
ENVIRONMENTAL True
MANAGEMENT True
CORP True

CAPITOL True
AUTOMOTIVE True
LLC False

EXTREME True
TERMITE True
PEST True
CONTROL True

THE True
COTTAGE True
SCHOOLEDUCATIONAL False
RESOURCE True
CENTER True
INC False

GOLTZ False
ASPHALT True
CO False

TRINITY True
INVESTIGATIONS False
SECURITY True
INC False

CLEAN True
AND True
SEAL True
CONSTRUCTION True
RESTORATION True
SERVICES False

CLEAN True
SEAL True
CONSTRUCTION True


In [0]:
# Compute recipe outputs from inputs
# TODO: Replace this part by your actual code that computes the output, as a Pandas dataframe
# NB: DSS also supports other kinds of APIs for reading and writing data. Please see doc.

WORD_LIST_df = BY_ACCOUNT_df # For this sample code, simply copy input to output


# Write recipe outputs
WORD_LIST = dataiku.Dataset("WORD_LIST")
WORD_LIST.write_with_schema(WORD_LIST_df)